This Jupyter Noteboox shows how to perform spatial univariate pre-whitening of GLM coefficients using the residual mean of squares (ResMS) output by SPM after GLM estimation. 

In [25]:
# relevant imports
import numpy as np
import rsatoolbox as rsa
from scipy.linalg import sqrtm
import nibabel as nb
import numpy as np

np.random.seed(7)


GLM parameters (betas) stored in .nii image files after SPM estimation can be loaded using nibabel.load() and then extracted as an i-by-j-by-k 3D tensor with the .get_fdata() method.

In [26]:
# betas = nb.load('path/to/glmDir/beta_0001.nii')

Alternatively, you can load betas from a specific ROI using world coordinates (x, y, z) with nitools.sample_image(). This may help you save space of disk. In this case, betas from the desired locations are stored as a vector.

In [27]:
# betas = nt.sample_image('path/to/glmDir/beta_0001.nii', x, y, z)

Here, we will simulate a vector of betas similar to the output of nitools.sample_image().

In [28]:
betas = np.random.random(size=(1000,))

Univariate pre-whitening involves normalizing the activity estimated in each voxel by the standard deviation of its residuals. SPM stores the residual mean squares in the ResMS.nii file in the same directory as the beta_XXXX.nii files. You can load them using nibabel.load('path/to/glmDir/ResMS.nii').get_fdata() or nitools.sample_image(). Here, we will simulate a vector of mean squares as done for the betas.

In [29]:
# load residual mean square
ResMS = np.random.random(size=(1000,))

To pre-whitening our betas, we now need to divide our betas by the square root of the residual mean squares.

In [30]:
betas_prewhitened = betas / np.sqrt(ResMS)

Calculating the euclidean distances between patterns of pre-whitened betas is equal to computing the mahalanobis distance using the residual mean squares instead of the covariance matrix. We can do as this as a sanity check that the pre-whitening was performed correctly. Let's first define a set of 5 betas each from a different condition.

In [31]:
betas = np.random.random(size=(5, 1000,))
betas_prewhitened = betas / np.sqrt(ResMS)
nCond = betas_prewhitened.shape[0]

We will now use the rsatoolbox to calculate the multivariate distances between conditions. First need to define a dataset object with the non-prewhitened and prewhitened betas

In [32]:
# create a dataset object
nVox = betas_prewhitened.shape[1]
des = {'session': 1, 'subj': 1}
obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}

# non-prewhitened dataset object
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

# prewhitened dataset object
dataset_prewhitened = rsa.data.Dataset(measurements=betas_prewhitened,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

Now we calculate the euclidean distances between prewhitened betas using the calc_rdm function in the rsatoolbox.

In [33]:
# calculate euclidean distance between conditions
dist_eucl = rsa.rdm.calc_rdm(dataset_prewhitened, method='euclidean', descriptor='conds')
print(dist.get_matrices())

[[[0.         1.08975407 1.17226038 0.92475594 1.15762617]
  [1.08975407 0.         1.06488425 0.93402951 0.93152588]
  [1.17226038 1.06488425 0.         1.15108832 1.38364892]
  [0.92475594 0.93402951 1.15108832 0.         0.99174954]
  [1.15762617 0.93152588 1.38364892 0.99174954 0.        ]]]


And the mahalanobis distance between non-prewhitened betas using the inverse of residual mean square in the noise argument. To do this, we need first to diagonalize the residual mean squares vector into a matrix.

In [34]:
# diagonalize ResMS
cov = np.diag(ResMS)

# calculate mahalanobis distance between conditions
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)
dist_ = rsa.rdm.calc_rdm(dataset, method='mahalanobis', descriptor='conds', noise=np.linalg.inv(cov))
print(dist.get_matrices())

[[[0.         1.08975407 1.17226038 0.92475594 1.15762617]
  [1.08975407 0.         1.06488425 0.93402951 0.93152588]
  [1.17226038 1.06488425 0.         1.15108832 1.38364892]
  [0.92475594 0.93402951 1.15108832 0.         0.99174954]
  [1.15762617 0.93152588 1.38364892 0.99174954 0.        ]]]


The two representational similarity matrices are numerically identical as expected.